# Training classifiers on HOG Data
#### For extraction of features, see extractFeatures.ipynb

conda env- faus_dl

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import sklearn
from sklearn import datasets, svm, metrics
import bcolz
import os, sys
import datetime as dt

In [2]:
DISFA_path='/media/amogh/Stuff/CMU/datasets/DISFA_data/'
DISFA_AU_path=DISFA_path+'ActionUnit_Labels/'
print(DISFA_AU_path)
Videos_right_path=DISFA_path+'Videos_RightCamera/'
print(Videos_right_path)

/media/amogh/Stuff/CMU/datasets/DISFA_data/ActionUnit_Labels/
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/


In [3]:
from mnist_helpers import *

Helper Functions

## FAU4_1

Loading the saved features:

In [4]:
dir_features_hog_1_fau4_1=DISFA_path+"/features/hog_1/fau4_1/"
dir_features_hog_1_fau4_1
if not os.path.exists(dir_features_hog_1_fau4_1):
    os.makedirs(dir_features_hog_hog_1_fau4_1)

In [5]:
features_fau4_1_hog_1_positives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'positives/',mode='r')
print("features_fau4_1_hog_1_positives loaded, shape is: ",features_fau4_1_hog_1_positives.shape)

features_fau4_1_hog_1_positives loaded, shape is:  (946, 448)


In [6]:
features_fau4_1_hog_1_negatives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'negatives/',mode='r')
print("features_fau4_1_hog_1_negatives loaded, shape is : ",features_fau4_1_hog_1_negatives.shape)

features_fau4_1_hog_1_negatives loaded, shape is :  (1000, 448)


Defining the training and testing data

In [7]:
X=np.concatenate([features_fau4_1_hog_1_positives,features_fau4_1_hog_1_negatives[:946]])
X.shape

(1892, 448)

In [8]:
Y=np.concatenate([np.repeat(1,features_fau4_1_hog_1_positives.shape[0]),np.repeat(0,features_fau4_1_hog_1_positives.shape[0])])
Y.shape

(1892,)

In [9]:
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,Y,test_size=0.15,random_state=42)

In [10]:
gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
gamma_range=gamma_range.flatten()
C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
C_range=C_range.flatten()
parameters={'kernel':['rbf'],'C':C_range,'gamma':gamma_range}

In [11]:
svm_clsf=svm.SVC()
grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=1,verbose=2)

In [12]:
start_time=dt.datetime.now()
print('Start param searching at {}'.format(str(start_time)))
grid_clsf.fit(X_train,y_train)
elapsed_time=dt.datetime.now()-start_time
print('Elapsed time, param searching {}'.format(str(elapsed_time)))
sorted(grid_clsf.cv_results_.keys())

Start param searching at 2018-05-27 13:09:50.635308
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   1.0s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   1.0s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   1.0s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   0.6s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   0.6s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   0.6s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   0.4s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   0.3s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed:  1.1min finished


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_gamma',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [13]:
classifier=grid_clsf.best_estimator_
params=grid_clsf.best_params_

scores=grid_clsf.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
plot_param_space_scores(scores,C_range, gamma_range)

NameError: name 'plot_param_space_scores' is not defined